In [1]:
!pip install tensortrade

     |████████████████████████████████| 133kB 6.8MB/s 
     |████████████████████████████████| 645kB 8.2MB/s 
     |████████████████████████████████| 788kB 9.5MB/s 
     |████████████████████████████████| 13.2MB 312kB/s 
     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 368kB 42.4MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.18 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.23.1 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
  Found existing installation: plotly

In [2]:
import pandas as pd
import tensortrade.env.default as default

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC, ETH
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.agents import DQNAgent


%matplotlib inline

In [3]:
cdd = CryptoDataDownload()

data = cdd.fetch("Bitstamp", "USD", "BTC", "1h")

In [4]:
def rsi(price: Stream[float], period: float) -> Stream[float]:
    r = price.diff()
    upside = r.clamp_min(0).abs()
    downside = r.clamp_max(0).abs()
    rs = upside.ewm(alpha=1 / period).mean() / downside.ewm(alpha=1 / period).mean()
    return 100*(1 - (1 + rs) ** -1)


def macd(price: Stream[float], fast: float, slow: float, signal: float) -> Stream[float]:
    fm = price.ewm(span=fast, adjust=False).mean()
    sm = price.ewm(span=slow, adjust=False).mean()
    md = fm - sm
    signal = md - md.ewm(span=signal, adjust=False).mean()
    return signal


features = []
for c in data.columns[1:]:
    s = Stream.source(list(data[c]), dtype="float").rename(data[c].name)
    features += [s]

cp = Stream.select(features, lambda s: s.name == "close")

features = [
    cp.log().diff().rename("lr"),
    rsi(cp, period=20).rename("rsi"),
    macd(cp, fast=10, slow=50, signal=5).rename("macd")
]

feed = DataFeed(features)
feed.compile()

In [5]:
bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(data["close"]), dtype="float").rename("USD-BTC")
)

portfolio = Portfolio(USD, [
    Wallet(bitstamp, 10000 * USD),
    Wallet(bitstamp, 10 * BTC)
])


renderer_feed = DataFeed([
    Stream.source(list(data["date"])).rename("date"),
    Stream.source(list(data["open"]), dtype="float").rename("open"),
    Stream.source(list(data["high"]), dtype="float").rename("high"),
    Stream.source(list(data["low"]), dtype="float").rename("low"),
    Stream.source(list(data["close"]), dtype="float").rename("close"), 
    Stream.source(list(data["volume"]), dtype="float").rename("volume") 
])


env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed,
    renderer_feed=renderer_feed,
    renderer=default.renderers.PlotlyTradingChart(),
    window_size=20
)

In [6]:
env.observer.feed.next()

{'external': {'lr': nan, 'macd': 0.0, 'rsi': nan},
 'internal': {'bitstamp:/BTC:/free': 10.0,
  'bitstamp:/BTC:/locked': 0.0,
  'bitstamp:/BTC:/total': 10.0,
  'bitstamp:/BTC:/worth': 87409.9,
  'bitstamp:/USD-BTC': 8740.99,
  'bitstamp:/USD:/free': 10000.0,
  'bitstamp:/USD:/locked': 0.0,
  'bitstamp:/USD:/total': 10000.0,
  'net_worth': 97409.9},
 'renderer': {'close': 8740.99,
  'date': Timestamp('2018-05-15 06:00:00'),
  'high': 8796.68,
  'low': 8707.28,
  'open': 8733.86,
  'volume': 559.93}}

In [9]:
!pip install pfrl

     |████████████████████████████████| 112kB 6.5MB/s 
  Created wheel for pfrl: filename=pfrl-0.2.1-cp37-none-any.whl size=153478 sha256=fb77ef26daa5926b948fcf32fe25dab9b28b49103ec44639860b48febe4cf424
  Stored in directory: /root/.cache/pip/wheels/10/22/cf/8da3465e0b3f7a915241dd5edd3d5d684ac62fb2ec57c25316
Successfully built pfrl


In [43]:
import pfrl


import numpy as np
import torch
from abc import ABCMeta, abstractmethod

from tensortrade.core import Identifiable

class QFunction(torch.nn.Module):

    def __init__(self, obs_size, n_actions):
        super().__init__()
        self.l1 = torch.nn.Linear(obs_size, 120)
        self.l2 = torch.nn.Linear(120, 120)
        self.l3 = torch.nn.Linear(120, n_actions)

    def forward(self, x):
        h = x
        h = torch.nn.functional.relu(self.l1(h))
        h = torch.nn.functional.relu(self.l2(h))
        h = self.l3(h)
        return pfrl.action_value.DiscreteActionValue(h)

class Agent(Identifiable, metaclass=ABCMeta):

    def __init__(self, env: 'TradingEnv'):
        self.env = env
        self.n_actions = env.action_space.n
        self.observation_shape = env.observation_space.shape


        
        self.dqn_agent = self.create_agent()
        self.env.agent_id = self.id

    def create_agent(self):
        flat_shape = self.observation_shape[0] * self.observation_shape[1]
       
        q_func = QFunction(flat_shape, self.n_actions)
        # Use Adam to optimize q_func. eps=1e-2 is for stability.
        optimizer = torch.optim.Adam(q_func.parameters(), eps=1e-2)

        # Set the discount factor that discounts future rewards.
        gamma = 0.9

        # Use epsilon-greedy for exploration
        explorer = pfrl.explorers.ConstantEpsilonGreedy(
            epsilon=0.3, random_action_func=self.env.action_space.sample)

        # DQN uses Experience Replay.
        # Specify a replay buffer and its capacity.
        replay_buffer = pfrl.replay_buffers.ReplayBuffer(capacity=10 ** 6)

        # Since observations from CartPole-v0 is numpy.float64 while
        # As PyTorch only accepts numpy.float32 by default, specify
        # a converter as a feature extractor function phi.
        phi = lambda x: x.astype(np.float32, copy=False)

        # Set the device id to use GPU. To use CPU only, set it to -1.
        gpu = -1

        # Now create an agent that will interact with the environment.
        agent = pfrl.agents.DoubleDQN(
            q_func,
            optimizer,
            replay_buffer,
            gamma,
            explorer,
            replay_start_size=500,
            update_interval=1,
            target_update_interval=100,
            phi=phi,
            gpu=gpu,
        )

        return agent

    
    def restore(self, path: str, **kwargs):
        """Restore the agent from the file specified in `path`."""
        pass

    
    def save(self, path: str, **kwargs):
        """Save the agent to the directory specified in `path`."""
        pass

    
    def get_action(self, state: np.ndarray, **kwargs) -> int:
        """Get an action for a specific state in the environment."""
        action = self.dqn_agent.act(state)
        return action

    
    def train(self,
              n_steps: int = None,
              n_episodes: int = 10000,
              save_every: int = None,
              save_path: str = None,
              callback: callable = None,
              **kwargs) -> float:
        """Train the agent in the environment and return the mean reward."""

        #n_episodes = 1000
        max_episode_len = 2**30
        for i in range(1, n_episodes + 1):
            obs = self.env.reset()
            t = 0  # time step
            while True:
                # Uncomment to watch the behavior in a GUI window
                # env.render()
                obs = obs.flatten()
                action = self.get_action(obs)
                obs, reward, done, _ = self.env.step(action)
                obs = obs.flatten()
                # print(action, reward, done)
                t += 1
                reset = t == max_episode_len
                self.dqn_agent.observe(obs, reward, done, reset)
                if done or reset:
                    break
            
            self.env.render()
        

        print('Finished.')



In [44]:
agent = Agent(env)

agent.train(n_steps=200, n_episodes=2, save_path="agents/")

FigureWidget({
    'data': [{'close': array([8740.99, 8739.  , 8728.49, ..., 4211.4 , 4209.18, 4130.62]),
    …

Finished.
